In [1]:
import sys
import numpy as np
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import *

store1=np.array([0,4,6,7,7,7,7])
store2=np.array([0,2,4,6,8,9,10])
store3=np.array([0,6,8,8,8,8,8])
store4=np.array([0,2,3,4,4,4,4])
table=(np.array([store1,store2,store3,store4])).T

store1=np.array([30,45,60,70,79,90,98,105,100,90])
store2=np.array([35,45,52,64,72,82,93,98,100,100])
store3=np.array([42,54,60,70,82,95,102,110,110,110])

zone1=np.array([1,2,3])
zone2=np.array([4,5,6])
table=(np.array([zone1,zone2])).T

In [2]:
class Ui_MainWindow(object):

    def __init__(self, max_zones, max_salesman, table, count):
        self.max_zones = max_zones
        self.max_salesman = max_salesman
        self.table = table
        self.count = count

    def solution(self,max_salesman, max_zones, table):
        result = []
        for i in range(0, max_salesman):
            result.append([])

        if max_zones == 1:
            return np.amax(table), np.where(table == np.amax(table))[0]

        elif max_zones == 2:
            first_zone = table[:, 0]
            second_zone = table[:, 1]
            intermediate_zone = first_zone + second_zone[::-1]
            max_i = np.where(intermediate_zone == np.amax(intermediate_zone))[0]
            for k in range(len(result)):
                result[k].append(str(k) + "+" + str(int(max_salesman - k - 1)))
            print("Intermediate_zone : \n", intermediate_zone, "\n")
            print("Result : \n", result, "\n")
            return np.amax(intermediate_zone), [result[i] for i, j in enumerate(intermediate_zone) if
                                                j == np.amax(intermediate_zone)]

        else:
            max_zones_uncovered = table.shape[1]
            dp = np.zeros((max_salesman, max_salesman))
            for i in range(0, max_salesman):
                for j in range(0, max_salesman):
                    if (i + j + 1 <= max_salesman):
                        dp[j, i] = table[i][0] + table[j][1]

            p = 0
            intermediate_zone = []
            for i in range(max_salesman):
                a, b = 0, p
                c, d = a, b
                l = []
                l_i = []
                for j in range(p + 1):
                    l.append(dp[a][b])
                    l_i.append((str(b) + "+" + str(a)))
                    a += 1
                    b -= 1
                intermediate_zone.append(max(l))
                max_i = np.where(l == np.amax(l))[0]
                for k in max_i:
                    result[p].append(l_i[k])
                p += 1

            max_zones_uncovered = max_zones_uncovered - 2
            print("Intermediate DP table : \n", dp, "\n")
            print("Intermediate_zone : \n", intermediate_zone, "\n")
            print("Result : \n", result, "\n")

            for k in range(0, max_zones_uncovered):
                if max_zones_uncovered == 1:
                    final_zone = table[:, max_zones - 1]
                    intermediate_zone = intermediate_zone + final_zone[::-1]
                    max_i = np.where(intermediate_zone == np.amax(intermediate_zone))[0]
                    for k in range(len(result)):
                        for m in range(len(result[k])):
                            result[k][m] = result[k][m] + "+" + str(int(max_salesman - k - 1))
                    print("Final step : \n")
                    print("Intermediate_zone : \n", intermediate_zone, "\n")
                    print("Result : \n", result, "\n")
                    return np.amax(intermediate_zone), [result[i] for i, j in enumerate(intermediate_zone) if
                                                        j == np.amax(intermediate_zone)]
                else:
                    result_t = []
                    for i in range(0, max_salesman):
                        result_t.append([])
                    dp = np.zeros((max_salesman, max_salesman))
                    for i in range(0, max_salesman):
                        for j in range(0, max_salesman):
                            if (i + j + 1 <= max_salesman):
                                dp[j, i] = intermediate_zone[i] + table[j][max_zones - max_zones_uncovered]
                    p = 0
                    intermediate_zone = []
                    for i in range(max_salesman):
                        a, b = 0, p
                        c, d = a, b
                        l = []
                        l_i = []
                        for j in range(p + 1):
                            l.append(dp[a][b])
                            l_i.append([a, b])
                            a += 1
                            b -= 1
                        intermediate_zone.append(max(l))
                        max_i = np.where(l == np.amax(l))[0]
                        for k in max_i:
                            for m in range(0, len(result[l_i[k][1]])):
                                result_t[p].append(result[l_i[k][1]][m] + "+" + str(l_i[k][0]))
                        p += 1
                    result = result_t
                    max_zones_uncovered = max_zones_uncovered - 1
                    print("intermediate DP table : \n", dp, "\n")
                    print("intermediate_zone : \n", intermediate_zone, "\n")
                    print("result : \n", result, "\n")

    def start_clicked(self):
        self.label_2.setHidden(False)
        self.label_3.setHidden(False)
        self.lineEdit.setHidden(False)
        self.lineEdit_2.setHidden(False)
        self.submit.setHidden(False)

    def submit_clicked(self):
        self.max_zones = int(self.lineEdit.text())
        self.max_salesman = int(self.lineEdit_2.text()) + 1
        self.label_2.setHidden(True)
        self.label_3.setHidden(True)
        self.lineEdit.setHidden(True)
        self.lineEdit_2.setHidden(True)
        self.submit.setHidden(True)
        self.table = np.empty((0,self.max_salesman), int)
        self.submit2.setHidden(False)
        self.label_5.setHidden(False)
        self.lineEdit_3.setHidden(False)

    def submit2_clicked(self):
        zone = self.lineEdit_3.text()
        zone = zone.split(',')
        zone = np.array([list(map(int, zone))])[:self.max_salesman]
        self.table = np.append(self.table, zone, axis=0)
        self.lineEdit_3.setText("")
        self.count = self.count+1
        if self.count == self.max_zones:
            self.label_5.setHidden(True)
            self.lineEdit_3.setHidden(True)
            self.submit2.setHidden(True)
            self.calculate.setHidden(False)
            self.table = np.transpose(self.table)
            print("Final Zones : ",self.max_zones)
            print("Final Salesman : ",self.max_salesman)
            print("Final table : ")
            print(self.table)
            self.label_6.setText("Number of Zones : "+str(self.max_zones))
            self.label_7.setText("Number of Salesman : "+str(self.max_salesman))
            #Row count
            self.tableWidget.setRowCount(self.max_salesman)
            self.tableWidget.horizontalHeader().setVisible(False)
            #Column count
            self.tableWidget.setColumnCount(self.max_zones)
            self.tableWidget.verticalHeader().setVisible(False)
            for i in range(self.max_salesman):
                for j in range(self.max_zones):
                    self.tableWidget.setItem(i,j, QTableWidgetItem(str(self.table[i][j])))
            
            self.tableWidget.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
            self.label_6.setHidden(False)
            self.label_7.setHidden(False)
            self.tableWidget.setHidden(False)  
            
    def calculate_clicked(self):
        profit,profit_combinations=self.solution(self.max_salesman,self.max_zones,self.table)
        print("Profit is : ",profit)
        print("Profit Combinations are : ",profit_combinations)
        q=QDialog()
        q.setWindowTitle("Result")
        q.resize(420,327)
        font = QtGui.QFont()
        font.setPointSize(12)
        label=QLabel(q)
        label.setGeometry(QtCore.QRect(120,0,231,41))
        label.setFont(font)
        label_text="Maximum Profit : " + str(profit)
        label.setText(label_text)
        textedit=QTextEdit(q)
        textedit.setGeometry(QtCore.QRect(130, 50, 151, 221))
        textedit.setFont(font)
        result_text=""
        for row in profit_combinations:
            for value in row:
                comb=value+"\n"
                result_text=result_text+comb
        textedit.setText(result_text)
        q.exec_()
        
        
    def quit_clicked(self):
        self.close()
        
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.setEnabled(True)
        MainWindow.resize(826, 532)
        MainWindow.setAutoFillBackground(False)
        MainWindow.setStyleSheet("")
        MainWindow.setTabShape(QtWidgets.QTabWidget.Rounded)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.start = QtWidgets.QPushButton(self.centralwidget)
        self.start.setGeometry(QtCore.QRect(20, 210, 93, 28))
        self.start.setStyleSheet("background-color: rgb(85, 255, 0);")
        self.start.setObjectName("start")
        self.start.clicked.connect(self.start_clicked)
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(410, 40, 155, 16))
        font = QtGui.QFont()
        font.setPointSize(10)
        font.setBold(True)
        font.setWeight(75)
        self.label.setFont(font)
        self.label.setFrameShape(QtWidgets.QFrame.NoFrame)
        self.label.setScaledContents(False)
        self.label.setObjectName("label")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setHidden(True)
        self.label_2.setGeometry(QtCore.QRect(230, 90, 161, 101))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.label_2.setFont(font)
        self.label_2.setObjectName("label_2")
        self.label_3 = QtWidgets.QLabel(self.centralwidget)
        self.label_3.setHidden(True)
        self.label_3.setGeometry(QtCore.QRect(230, 270, 201, 51))
        font = QtGui.QFont()
        font.setPointSize(12)
        self.label_3.setFont(font)
        self.label_3.setObjectName("label_3")
        self.lineEdit = QtWidgets.QLineEdit(self.centralwidget)
        self.lineEdit.setHidden(True)
        self.lineEdit.setGeometry(QtCore.QRect(480, 130, 171, 22))
        self.lineEdit.setObjectName("lineEdit")
        self.lineEdit_2 = QtWidgets.QLineEdit(self.centralwidget)
        self.lineEdit_2.setHidden(True)
        self.lineEdit_2.setGeometry(QtCore.QRect(480, 290, 171, 22))
        self.lineEdit_2.setObjectName("lineEdit_2")
        self.picture = QtWidgets.QLabel(self.centralwidget)
        self.picture.setGeometry(QtCore.QRect(380, 380, 131, 51))
        self.picture.setText("")
        self.picture.setObjectName("picture")
        self.label_4 = QtWidgets.QLabel(self.centralwidget)
        self.label_4.setHidden(True)
        self.label_4.setGeometry(QtCore.QRect(140, 10, 671, 441))
        self.label_4.setAutoFillBackground(False)
        self.label_4.setFrameShadow(QtWidgets.QFrame.Plain)
        self.label_4.setText("")
        self.label_4.setPixmap(QtGui.QPixmap("bgimg.jpg"))
        self.label_4.setScaledContents(True)
        self.label_4.setHidden(False)
        self.label_4.setObjectName("label_4")
        self.submit = QtWidgets.QPushButton(self.centralwidget)
        self.submit.setGeometry(QtCore.QRect(410, 400, 93, 28))
        self.submit.setStyleSheet("background-color: rgb(85, 255, 0);")
        self.submit.setObjectName("submit")
        self.submit.setHidden(True)
        self.submit.clicked.connect(self.submit_clicked)
        self.submit2 = QtWidgets.QPushButton(self.centralwidget)
        self.submit2.setGeometry(QtCore.QRect(410, 400, 93, 28))
        self.submit2.setStyleSheet("background-color: rgb(85, 255, 0);")
        self.submit2.setObjectName("submit2")
        self.submit2.setHidden(True)
        self.submit2.clicked.connect(self.submit2_clicked)
        self.calculate = QtWidgets.QPushButton(self.centralwidget)
        self.calculate.setGeometry(QtCore.QRect(410, 400, 93, 28))
        self.calculate.setStyleSheet("background-color: rgb(85, 255, 0);")
        self.calculate.setObjectName("calculate")
        self.calculate.setHidden(True)
        self.calculate.clicked.connect(self.calculate_clicked)
        self.label_5 = QtWidgets.QLabel(self.centralwidget)
        self.label_5.setHidden(True)
        self.label_5.setGeometry(QtCore.QRect(230, 200, 201, 51))
        font = QtGui.QFont()
        font.setPointSize(12)
        font.setBold(False)
        font.setWeight(50)
        self.label_5.setFont(font)
        self.label_5.setObjectName("label_5")
        self.label_6 = QtWidgets.QLabel(self.centralwidget)
        self.label_6.setHidden(True)
        self.label_6.setGeometry(QtCore.QRect(210, 70, 231, 31))
        self.label_6.setFont(font)
        self.label_6.setObjectName("label_6")
        self.label_7 = QtWidgets.QLabel(self.centralwidget)
        self.label_7.setHidden(True)
        self.label_7.setGeometry(QtCore.QRect(490, 70, 291, 31))
        self.label_7.setFont(font)
        self.label_7.setObjectName("label_7")
        self.lineEdit_3 = QtWidgets.QLineEdit(self.centralwidget)
        self.lineEdit_3.setHidden(True)
        self.lineEdit_3.setGeometry(QtCore.QRect(480, 220, 171, 22))
        self.lineEdit_3.setObjectName("lineEdit_3")
        self.tableWidget = QTableWidget(self.centralwidget)
        self.tableWidget.setGeometry(QtCore.QRect(300,130, 311, 231))
        self.tableWidget.setFont(font)
        self.tableWidget.setHidden(True)
        self.label_4.raise_()
        self.start.raise_()
        self.label.raise_()
        self.label_2.raise_()
        self.label_3.raise_()
        self.lineEdit.raise_()
        self.lineEdit_2.raise_()
        self.picture.raise_()
        self.submit.raise_()
        self.submit2.raise_()
        self.calculate.raise_()
        self.label_5.raise_()
        self.label_6.raise_()
        self.label_7.raise_()
        self.lineEdit_3.raise_()
        self.tableWidget.raise_()
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 826, 26))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)
        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.start.setText(_translate("MainWindow", "Start"))
        self.start.setShortcut(_translate("MainWindow", "S"))
        self.label.setText(_translate("MainWindow", "PROJECT"))
        self.label_2.setText(_translate("MainWindow", "Number of Zones"))
        self.label_3.setText(_translate("MainWindow", "Number of Salesman"))
        self.submit.setText(_translate("MainWindow", "Submit"))
        self.submit.setShortcut(_translate("MainWindow", "Return"))
        self.submit2.setText(_translate("MainWindow", "Submit"))
        self.submit2.setShortcut(_translate("MainWindow", "Return"))
        self.calculate.setText(_translate("MainWindow", "Calculate"))
        self.calculate.setShortcut(_translate("MainWindow", "Return"))
        self.label_5.setText(_translate("MainWindow", "Enter Zone Details"))
        self.label_6.setText(_translate("MainWindow", "Number of Zones : "))
        self.label_7.setText(_translate("MainWindow", "Number of Salesman : "))

In [3]:
if __name__ == "__main__":
    global max_zones
    max_zones = 0
    global max_salesman
    max_salesman =0
    global table
    table = np.array([])
    global count
    count = 0
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow(max_zones, max_salesman, table, count)
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

Final Zones :  2
Final Salesman :  3
Final table : 
[[1 4]
 [2 5]
 [3 6]]
Intermediate_zone : 
 [7 7 7] 

Result : 
 [['0+2'], ['1+1'], ['2+0']] 

Profit is :  7
Profit Combinations are :  [['0+2'], ['1+1'], ['2+0']]


SystemExit: 0

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
